In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error

In [2]:
import mlflow
mlflow.set_tracking_uri("$MLFLOW_DB_URI")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/imvaibhavrana/mlops/02-experiment-tracking/$MLFLOW_DB_URI/886638330577656943', creation_time=1757386861713, experiment_id='886638330577656943', last_update_time=1757386861713, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [6]:
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']
target = 'duration'
def get_df(parquet_file_path):
    df = pd.read_parquet(parquet_file_path)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df["duration"] = df["duration"].dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <=60)]
    df[categorical] = df[categorical].astype(str)
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']   # creating a new feature
    return df

In [10]:
train_df = get_df("data/yellow_tripdata_2023-01.parquet")
val_df = get_df("data/yellow_tripdata_2023-02.parquet")
train_df = train_df.head(100000)
val_df = val_df.head(100000)

In [11]:
dv = DictVectorizer()
new_feature = ['PU_DO']
train_dict = train_df[new_feature + numerical].to_dict(orient='records')
val_dict = val_df[new_feature + numerical].to_dict(orient='records')

x_train = dv.fit_transform(train_dict)
y_train = train_df[target].values

x_val = dv.transform(val_dict)
y_val = val_df[target].values

In [ ]:
with mlflow.start_run():
    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    mlflow.set_tag('model', 'Ridge')
    model = Ridge(alpha=alpha, random_state=42)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    print(f"RMSE: {rmse}")
    with open('model.bin','wb') as f_out:
        pickle.dump((dv,model),f_out)